In [1]:
# 導入必要的庫
import pandas as pd
import json
from datetime import timedelta

# 讀取 JSON 檔案
file_path = '/Users/chianlee/Desktop/disney/data/land_data.json'
file_path_names = '/Users/chianlee/Desktop/disney/data/land_namelist.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

with open(file_path_names, 'r', encoding='utf-8') as file:
    names = json.load(file)

# 轉換為 DataFrame
df_data = pd.DataFrame(data)
df_names = pd.DataFrame(names)

# 刪除 'FacilityName' 欄位並移除 'StandbyTime' 為 null, false 的行
df_data = df_data.drop(columns=['FacilityName'])
df_data = df_data[df_data['StandbyTime'].notna()]
df_data = df_data.dropna()  # 移除包含 NaN 的行
df_data = df_data[df_data['StandbyTime'] != False]

# 解析日期時間欄位
df_data['UpdateTime'] = pd.to_datetime(df_data['UpdateTime'], format='%H:%M')
df_data['datetime'] = pd.to_datetime(df_data['datetime'])

# 將所有 datetime 欄位加上一小時
df_data['datetime'] = df_data['datetime'] + pd.Timedelta(hours=1)

# 提取日期並應用於 UpdateTime
df_data['UpdateTime'] = df_data.apply(
    lambda row: row['UpdateTime'].replace(year=row['datetime'].year, month=row['datetime'].month, day=row['datetime'].day),
    axis=1
)

# 計算 UpdateTime 和 datetime 之間的時間差（以小時為單位）
df_data['time_difference'] = (df_data['datetime'] - df_data['UpdateTime'])

# 刪除時間差超過2小時的資料
df_filtered = df_data[df_data['time_difference'] <= timedelta(hours=2)]

# 刪除多餘的時間差欄位
df_filtered = df_filtered.drop(columns=['time_difference'])

# 將 StandbyTime 轉換為數值型
df_filtered['StandbyTime'] = pd.to_numeric(df_filtered['StandbyTime'], errors='coerce')

# 提取設施名稱、日期和小時
df_filtered['Hour'] = df_filtered['datetime'].dt.hour
df_filtered['Date'] = df_filtered['datetime'].dt.date

# 篩選 8 點到 20 點之間的數據
df_filtered = df_filtered[(df_filtered['Hour'] >= 8) & (df_filtered['Hour'] <= 20)]

# 合併數據框以添加英文設施名稱
df_filtered = df_filtered.merge(df_names[['FacilityID', 'FacilityEnglish']], on='FacilityID', how='left')

# 確保每個設施在每個日期的 8 點到 20 點都有數據
all_hours = pd.DataFrame({'Hour': range(8, 21)})
all_dates = df_filtered['Date'].unique()
all_facilities = df_filtered['FacilityEnglish'].unique()

full_df = pd.DataFrame()
for date in all_dates:
    for facility in all_facilities:
        facility_data = pd.merge(all_hours, df_filtered[(df_filtered['FacilityEnglish'] == facility) & (df_filtered['Date'] == date)], on='Hour', how='left')
        facility_data['FacilityEnglish'] = facility
        facility_data['Date'] = date
        
        # 對該設施在該日期的 StandbyTime 進行線性插值和前後填充
        facility_data['StandbyTime'] = facility_data['StandbyTime'].interpolate(method='linear').ffill().bfill()
        full_df = pd.concat([full_df, facility_data], ignore_index=True)

# 計算每個設施每個日期每個小時的平均等待時間
average_wait_times_per_day_hour = full_df.groupby(['FacilityEnglish', 'Date', 'Hour'])['StandbyTime'].mean().reset_index()

# 確保 Date 欄位以 ISO 8601 格式存儲
average_wait_times_per_day_hour['Date'] = average_wait_times_per_day_hour['Date'].astype(str)

# 將結果存成 JSON 格式
output_file_path = '/Users/chianlee/Desktop/disney/data/tableau_data_land.json'

# 轉換 DataFrame 為 JSON
average_wait_times_per_day_hour.to_json(output_file_path, orient='records', lines=True, force_ascii=False, date_format='iso')


In [7]:
import pandas as pd
import json

# 讀取 JSON 檔案
file_path = '/Users/chianlee/Desktop/disney/data/tableau_data_land.json'

# 轉換為 DataFrame
df_tableau = pd.read_json(file_path, lines=True)

# 計算 StandbyTime 欄位中的 null 值數量
null_count = df_tableau['StandbyTime'].isnull().sum()

print(f"StandbyTime 欄位中共有 {null_count} 個 null 值。")


StandbyTime 欄位中共有 0 個 null 值。


In [8]:
# 篩選出 StandbyTime 欄位為 null 的行
null_standbytime_df = df_tableau[df_tableau['StandbyTime'].isnull()]

# 找出這些行所對應的設施名稱
facilities_with_null_standbytime = null_standbytime_df['FacilityEnglish'].unique()

print("StandbyTime 欄位中出現 null 值的設施包括：")
for facility in facilities_with_null_standbytime:
    print(facility)


StandbyTime 欄位中出現 null 值的設施包括：
